# What happens:

File is created and email is prepared

## Import 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import datetime
import win32com.client as win32
import calendar
from openpyxl import Workbook, load_workbook
from IPython.lib.display import Audio
from datetime import date

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [2]:
directory = os.path.abspath(os.getcwd()) + "/"
rd = str(input("Please enter the RD (YYYYMMDD): "))
ws = "0194"

### Make connection with Oracle DB

In [3]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

### Select per Unit

In [4]:

file_path = "RAY-extract-Template.xlsx"
im_folder = "Import/"
ex_folder = "Export/"
today = date.today()
date_today = today.strftime("%d.%m.%Y")

# Generating workbook
ExcelWorkbook = load_workbook(filename = im_folder + file_path)
 
# Generating the writer engine
file_date = rd[6:8] + "." + rd[4:6] + "." + rd[:4]
file_name = "RAY-Extract-" + date_today + "_data from_" + file_date + ".xlsx"
writer = pd.ExcelWriter(ex_folder + file_name, engine = 'openpyxl')
 
# Assigning the workbook to the writer engine
writer.book = ExcelWorkbook
 
units = ['RBAL', 'RBRS', 'RBBH', 'RBKO', 'RBBY', 'AVAL', 'RBBG', 'RBHR']
for unit in units:
    print(unit + " started")
    
    d1 = pd.read_sql_query("""select reporting_date,counterparty_code,counterparty_name,counterparty_new_code,counterparty_new_name,contract_type,contract_type_desc, contract_reference, attribute_5, maturity_date, tenor_original, tenor_residual, currency, ead_pre_ccf, ead, rwa 	
    from rzb_cdr 	
    where booking_company='""" + unit + """' and ead_pre_ccf >0 and counterparty_type in ('12', '30')	
    --group by reporting_date,counterparty_code,counterparty_name,counterparty_new_code,counterparty_new_name,contract_type,contract_type_desc,currency	
    """, connection_RAY) 

    d1.to_excel(writer, sheet_name = unit, index=False)
    print(unit + " ended")
    
print("Equity started")
d1 = pd.read_sql_query("""select * from rzb_cdr where counterparty_code in ('2701'	
,'15095'	
,'27841'	
,'27889'	
,'41771'	
,'39888') and record_source = 'T_CDR' and table_name = 'EQUITY'""", connection_RAY) 

d1.to_excel(writer, sheet_name = "EQUITY", index=False)
print("Equity ended")

ExcelWorkbook.save(ex_folder + file_name)
ExcelWorkbook.close()

C:\Users\WZHARBC\AppData\Local\Temp\1\ipykernel_17716\2261522452.py:16: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = ExcelWorkbook


RBAL started
RBAL ended
RBRS started
RBRS ended
RBBH started
RBBH ended
RBKO started
RBKO ended
RBBY started
RBBY ended
AVAL started
AVAL ended
RBBG started
RBBG ended
RBHR started
RBHR ended
Equity started
Equity ended


In [5]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'oleg.lenski@rbinternational.com; '
mail.Cc = 'markus.hochleitner@rbinternational.com; nertila.gjakova@rbinternational.com'
mail.Subject = 'Central Bank Exposure ' + file_date
mail.Body = '''Dear Oleg,

Please find attached the data for the cut-off date ''' + file_date + '''

Kind regards,
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''

# To attach a file to the email (optional):
attachment  = directory + ex_folder + file_name
mail.Attachments.Add(attachment)

mail.Display()